In [27]:
COST_INDEX = 0.07
SELL_TAX = 0.036
LOCATION = 60003760
REGION = 10000002
MAN_DURATION_MOD = (1 - 0.32) * (1 - 0.15)
INV_DURATION_MOD = (1 - 0.15) * (1 - 0.15)
COPY_DURATION_MOD = (1 - 0.25) * (1 - 0.15) * (1 - 0.15) * (1 - 0.2)

In [28]:
# Collect data from database
from src.util import connect_to_db
from collections import defaultdict
from decimal import Decimal

conn = await connect_to_db()

type_ids = dict(await conn.fetch('SELECT type_id, name FROM sde.type_ids'))
type_names = dict(await conn.fetch('SELECT name, type_id FROM sde.type_ids'))
type_published = dict(await conn.fetch('SELECT type_id, published FROM sde.type_ids'))

bp_info = await conn.fetch('SELECT * FROM sde.blueprints')
bp_info = {bp['blueprint_type_id']: bp for bp in bp_info}

bp_inv_mat_data = await conn.fetch("SELECT * FROM sde.blueprint_materials WHERE activity = 'invention'")
bp_inv_mats = defaultdict(list)
for bp_mat in bp_inv_mat_data:
    bp_inv_mats[bp_mat["blueprint_type_id"]].append({'type_id': bp_mat['material_type_id'], 'quantity': bp_mat['quantity']})
    
bp_man_mat_data = await conn.fetch("SELECT * FROM sde.blueprint_materials WHERE activity = 'manufacturing'")
bp_man_mats = defaultdict(list)
for bp_mat in bp_man_mat_data:
    bp_man_mats[bp_mat["blueprint_type_id"]].append({'type_id': bp_mat['material_type_id'], 'quantity': bp_mat['quantity']})

bp_inv_prod_data = await conn.fetch("SELECT * FROM sde.blueprint_products WHERE activity = 'invention'")
bp_man_prod_data = await conn.fetch("SELECT * FROM sde.blueprint_products WHERE activity = 'manufacturing'")
bp_man_prods = {}
for bp_prod in bp_man_prod_data:
    bp_man_prods[bp_prod["blueprint_type_id"]] = {'type_id': bp_prod['product_type_id'], 'quantity': bp_prod['quantity']}
    
market_buy = dict(await conn.fetch(f'SELECT type_id, buy_max FROM market.aggregates WHERE location_id = {LOCATION}'))
market_sell = dict(await conn.fetch(f'SELECT type_id, sell_min FROM market.aggregates WHERE location_id = {LOCATION}'))
market_region_sell = dict(await conn.fetch(f'SELECT type_id, MIN(sell_min) FROM market.aggregates WHERE region_id = {REGION} GROUP BY type_id'))

In [29]:
SELL_TAX = Decimal(SELL_TAX)
COST_INDEX = Decimal(COST_INDEX)

invention_price = defaultdict(int)
avg_invention_price = defaultdict(int)
buy_price = defaultdict(int)
sell_price = defaultdict(int)
profit = defaultdict(int)
profit_per_day = defaultdict(int)
profitable_price = defaultdict(int)
manufacture_time = defaultdict(int)

for bp_prod in bp_inv_prod_data:
    original_bp_id = bp_prod['blueprint_type_id']
    bp_id = bp_prod['product_type_id']
    
    if type_published[bp_id] == False or type_published[original_bp_id] == False:
        continue
    
    cant_buy_materials = False
    for inv_mat in bp_inv_mats[original_bp_id]:
        if inv_mat['type_id'] not in market_sell or market_sell[inv_mat['type_id']] is None:
            cant_buy_materials = True
            break
        invention_price[bp_id] += market_sell[inv_mat['type_id']] * inv_mat['quantity']
    avg_invention_price[bp_id] = invention_price[bp_id] / bp_prod["probability"]
    
    if cant_buy_materials:
        continue
    
    cant_buy_materials = False
    for man_mat in bp_man_mats[bp_id]:
        if man_mat['type_id'] not in market_sell or market_sell[man_mat['type_id']] is None:
            cant_buy_materials = True
            break
        buy_price[bp_id] += market_sell[man_mat['type_id']] * man_mat['quantity'] * bp_info[bp_id]['max_production_limit']
        
    if cant_buy_materials:
        continue
    
    sell_price[bp_id] = (market_buy.get(bp_man_prods[bp_id]['type_id'], Decimal(0)) or 0) * bp_man_prods[bp_id]['quantity'] * bp_info[bp_id]['max_production_limit']
    # sell_price[bp_id] = (market_sell.get(bp_man_prods[bp_id]['type_id'], 0) or 0) * bp_man_prods[bp_id]['quantity']
    
    profitable_price[bp_id] = (avg_invention_price[bp_id] + buy_price[bp_id] / (1 - COST_INDEX) / (1 - SELL_TAX)) / bp_man_prods[bp_id]['quantity']
    
    manufacture_time[bp_id] += (bp_info[original_bp_id]['copying_time'] or 0) * Decimal(COPY_DURATION_MOD) / bp_prod["probability"]
    manufacture_time[bp_id] += bp_info[original_bp_id]['invention_time'] * Decimal(INV_DURATION_MOD) / bp_prod["probability"]
    manufacture_time[bp_id] += bp_info[bp_id]['manufacturing_time'] * Decimal(MAN_DURATION_MOD) * bp_info[bp_id]['max_production_limit']
    
    profit[bp_id] = sell_price[bp_id] - buy_price[bp_id] - sell_price[bp_id] * SELL_TAX - buy_price[bp_id] * COST_INDEX - avg_invention_price[bp_id]
    profit_per_day[bp_id] = profit[bp_id] * Decimal(86400 / manufacture_time[bp_id])

In [30]:
# Print out profit for purchasable BPOs
print(f"{'Name':<64} {'Profit/day':>16} {'Profit/run':>16} {'Buy price':>16} {'Sell price':>16}")
for bp_id, bp_profit in sorted(profit_per_day.items(), key=lambda item: item[1], reverse=True):
    print(f"{type_ids[bp_id]:<64} {profit_per_day[bp_id]:>16,.0f} {profit[bp_id]:>16,.0f} {buy_price[bp_id]:>16,.0f} {sell_price[bp_id]:>16,.0f}")

Name                                                                   Profit/day       Profit/run        Buy price       Sell price
Standup Focused Warp Disruptor II Blueprint                            30,578,441      162,881,161      687,932,780      939,000,000
Exceptional Moon Mining Crystal Type B II Blueprint                    27,092,789        3,164,871       10,821,922       19,870,000
Small Remote Hull Repairer II Blueprint                                19,581,515        9,919,533       10,135,960       22,110,000
Uncommon Moon Mining Crystal Type B II Blueprint                       15,689,761        1,832,815        8,529,552       14,800,000
Standup Point Defense Battery II Blueprint                             13,383,513       71,289,514    1,619,321,565    1,877,000,000
Barrage XL Blueprint                                                   10,398,983       40,823,711       76,889,000      128,250,000
Standup Stasis Webifier II Blueprint                                 